年报解析环境信息

1.显著性
    - 1分：仅非财务部分披露环境信息
    - 2分：仅财务部分披露环境信息
    - 3分：财务、非财务部分都披露环境信息


2.量化性
    - 1分：仅文字描述
    - 2分：数量但非货币信息
    - 3分：货币化信息


3.时间性
    - 1分：仅过去信息
    - 2分：仅未来信息
    - 3分：现在与过去对比信息

In [40]:
import numpy as np
import pandas as pd
import requests, os, json
from bs4 import BeautifulSoup

In [51]:
from Proxy import get_proxy, getWeb, gen_proxy

In [52]:
proxies = get_proxy()

成功获取代理！


In [49]:
txtPath = "./txt/"
csvPath = "./年度报告/"

In [99]:
code = "601899"

In [ ]:
# 解析 网易财经 定期报告表格，获取年报链接
url = "http://quotes.money.163.com/f10/gsgg_601899,dqbg.html"
soupTable = getWeb(url, proxies=proxies, Return="soup")
#soupTable = BeautifulSoup(responseTable)

In [104]:
result = soupTable.find_all("div", class_="tabs_panel")
res = result[0].get_text().split("\n")
res = list(filter(None, res))[:-1]
href = ["http://quotes.money.163.com"+item.get("href") for item in result[0].find_all("a")][:-3]
npRes = np.array(res).reshape(-1, 3)
df = pd.DataFrame(npRes[1:, :], columns=npRes[0, :])
df["href"] = href
df["releaseYear"] = df["公布日期"].map(lambda x: x[:4])
index = df["公告类型"]=="年度报告"
df = df[index]

In [113]:
corp = "紫金矿业"

In [115]:
df.to_csv("{}{}_{}.csv".format(csvPath, code, corp), encoding="gbk", index=False)

In [105]:
df.shape

(5, 5)

In [106]:
df

,公告标题,公布日期,公告类型,href,releaseYear
4,紫金矿业2017年年度报告,2018-03-24,年度报告,http://quotes.money.163.com/f10/ggmx_601899_41...,2018
9,紫金矿业2016年年度报告,2017-03-25,年度报告,http://quotes.money.163.com/f10/ggmx_601899_31...,2017
16,紫金矿业H股公告-2015年报,2016-04-30,年度报告,http://quotes.money.163.com/f10/ggmx_601899_24...,2016
18,紫金矿业年报,2016-03-26,年度报告,http://quotes.money.163.com/f10/ggmx_601899_22...,2016
22,紫金矿业集团股份有限公司年报,2015-03-21,年度报告,http://quotes.money.163.com/f10/ggmx_601899_16...,2015


In [121]:
from multiprocessing import Pool

In [ ]:
pool = Pool()
temp = pool.imap_unordered(get_write, range(len(df)))
texts = [item for item in temp]
pool.close()

In [126]:
# 获取每年年报，并写入 txt 文件
# i = 1
def get_write(i):
    '''
    获取每年年报，并写入 txt 文件
    :param i: 年度报告列表第几个年报
    '''
    text = getWeb(df.href.iloc[i], Return="text", proxies=proxies)
    text = text.replace("&ensp;", "")
    # text = text.replace("\r<br/>", "")
    txtFile = r"{}{}_{}.txt".format(txtPath, code, df["公告标题"].iloc[i])
    with open(txtFile, "w") as f:
        f.write(text)
    return text

In [1]:
text

NameError: name 'text' is not defined

In [19]:
with open("./txt/601899_紫金矿业2017年年度报告.txt", "r", encoding="utf8") as f:
    text = f.read()

In [20]:
s = text[:1000]

In [21]:
s

'<html>\n<head>\n    <title>紫金矿业2017年年度报告</title>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n    <meta name="keywords" content="公告明细"/>\n    <meta name="description" content="紫金矿业2017年年度报告"/>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n    <meta http-equiv="Content-Language" content="zh-CN"/>\n    <meta name="robots" content="index, follow"/>\n    <meta name="googlebot" content="index, follow"/>\n    <link href="http://img1.cache.netease.com/f2e/finance/gegu/s.1064000.css" rel="stylesheet" type="text/css" charset="utf-8">\n<link href="http://img1.cache.netease.com/f2e/finance/gegu/danmaku.959699.css" rel="stylesheet" type="text/css" charset="utf-8">\n</head>\n<body>\n<div class="report_detail">\n    <h1 class="title_01">\n        <span class="name">紫金矿业2017年年度报告</span>\n    </h1>\n\n    <div class="inner_box" id="newsTabs">\n        <div class="report_detail">\n            <div class="inner_box">\n                公司代码：60189

# 解析网易财经年报目录

In [75]:
def parseContent(text):
    '''
    用于解析年报目录，返回 df
    :param text: 年报内容，str 类型
    '''
#     content = text[text.find("目录"):text.find("目录")+3000]
    info = text[text.find("<title>")+len("<title>"):text.find("</title>")]
    content = text[text.find(".......")-100:text.find(".......")+3000]
    temp = content[content.find("目录")+2:content.find("<br/>", content.rfind(".."))].replace("&ensp;", "").replace("<br/>", "").strip().split()
    dfContent = pd.DataFrame([list(filter(None, item.split("."))) for item in temp], columns=["title", "page"])
    dfContent["section"] = dfContent.title.map(lambda x: x[:x.find("节")+1])
    dfContent["sectiontitle"] = dfContent.title.map(lambda x: x[x.find("节")+1:])
    dfContent["info"] = info
    return dfContent

In [76]:
parseContent(text)

,title,page,section,sectiontitle,info
0,第一节释义,4,第一节,释义,紫金矿业2017年年度报告
1,第二节公司简介和主要财务指标,4,第二节,公司简介和主要财务指标,紫金矿业2017年年度报告
2,第三节公司业务概要,8,第三节,公司业务概要,紫金矿业2017年年度报告
3,第四节经营情况讨论与分析,9,第四节,经营情况讨论与分析,紫金矿业2017年年度报告
4,第五节重要事项,30,第五节,重要事项,紫金矿业2017年年度报告
5,第六节普通股股份变动及股东情况,47,第六节,普通股股份变动及股东情况,紫金矿业2017年年度报告
6,第七节优先股相关情况,53,第七节,优先股相关情况,紫金矿业2017年年度报告
7,第八节董事、监事、高级管理人员和员工情况,54,第八节,董事、监事、高级管理人员和员工情况,紫金矿业2017年年度报告
8,第九节公司治理,61,第九节,公司治理,紫金矿业2017年年度报告
9,第十节公司债券相关情况,64,第十节,公司债券相关情况,紫金矿业2017年年度报告


In [71]:
# url = "http://quotes.money.163.com/f10/ggmx_002594_4159834.html" # 比亚迪
url = "http://quotes.money.163.com/f10/ggmx_600188_4147752.html" # 兖州煤业
response = getWeb(url, proxies=proxies, Return="text")

第1次代理成功


In [77]:
parseContent(response)

,title,page,section,sectiontitle,info
0,第一节释义,4,第一节,释义,兖州煤业2017年年度报告
1,第二节公司简介和主要财务指标,7,第二节,公司简介和主要财务指标,兖州煤业2017年年度报告
2,第三节公司业务概要,13,第三节,公司业务概要,兖州煤业2017年年度报告
3,第四节董事长报告书,14,第四节,董事长报告书,兖州煤业2017年年度报告
4,第五节经营情况讨论与分析,17,第五节,经营情况讨论与分析,兖州煤业2017年年度报告
5,第六节重要事项,39,第六节,重要事项,兖州煤业2017年年度报告
6,第七节普通股股份变动及股东情况,72,第七节,普通股股份变动及股东情况,兖州煤业2017年年度报告
7,第八节董事、监事、高级管理人员和员工情况,79,第八节,董事、监事、高级管理人员和员工情况,兖州煤业2017年年度报告
8,第九节公司治理,90,第九节,公司治理,兖州煤业2017年年度报告
9,第十节公司债券相关情况,102,第十节,公司债券相关情况,兖州煤业2017年年度报告


In [80]:
import re

In [82]:
# 搜索全文包含 环境 前后50个字符
place = [item.start() for item in re.finditer("环境", text)]

In [88]:
t = [text[item-50:item+50] for item in place]

In [89]:
t

['位。\n<br/>第四节经营情况讨论与分析\n<br/>一、经营情况讨论与分析\n<br/>报告期外部经营环境\n<br/>2017年全球发达经济体稳步复苏,中国经济稳中向好，经济增长好于预期；美联储加息和缩',
 '极把握多个海外项目并购机会，加快国际化发展步伐。\n<br/>13/70\n<br/>坚持矿业开发与生态环境高度统一，积极履行社会责任\n<br/>报告期，公司全面按照新时期绿色发展要求，以“零工亡、零职业',
 '币。\n<br/>(二)非主营业务导致利润重大变化的说明\n<br/>√适用□不适用\n<br/>根据外部环境的变化和相关子公司的经营现状，为了更加客观公允反映公司的财务状况和资\n<br/>产价值，根据《企',
 '做强做大的重要依\n<br/>托。\n<br/>(三)经营计划\n<br/>√适用□不适用\n<br/>经营环境\n<br/>黄金是经过历史检验的、不可替代的全球性战略资产，是各国金融储备体系的基石，在维护\n<',
 '部为主的\n<br/>各板块服务和管理功能，重点提升财务、物流、人力资源等业务协同增值。把“安全生产、环境\n<br/>保护、生态发展、依法合规、可持续发展”作为公司发展的五大原则，提升发展效率和质量。\n',
 '度重视安全环保、职业健康和生态建设，树立行业标杆形象\n<br/>贯彻新时代的发展要求，狠抓安全生产和环境保护，打造行业绿色发展新标杆，把安全环保\n<br/>和生态建设打造成公司的品牌，尤其是公司生态建',
 '，综合运用制度、\n<br/>管理、经济等手段，确保集团公司安全生产持续、稳定；公司高度重视并持续改进环境保护工作，\n<br/>践行“要金山银山，更要绿水青山”的环保理念，切实做好环境保护、生态修复，大',
 '公司高度重视并持续改进环境保护工作，\n<br/>践行“要金山银山，更要绿水青山”的环保理念，切实做好环境保护、生态修复，大力发展绿色\n<br/>矿山建设，努力推动形成绿色发展格局。\n<br/>4.国家',
 '责。公司通过对贫困户和贫困村精准识别、精准帮扶，着力改善贫困村基\n<br/>础设施条件和贫困群众居住环境，稳步提高贫困群众生活水平，并逐步构建公司精准扶贫工作长\n<br/>效机制。\n<br/>42/7',
 '古、黑龙江、云南、青海、山西等省（自治区）开展

In [79]:
text.find("环境")

10193

In [100]:
import urllib

In [102]:
urllib.parse.quote("中")

'%E4%B8%AD'

In [111]:
import requests

headers = {
    'origin': 'https://goldenbee.applysquare.com',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'x-requested-with': 'XMLHttpRequest',
    'cookie': 'UM_distinctid=165f4d269c367d-0c70e71d1ccba6-323b5b03-144000-165f4d269c4a29; Hm_lvt_50058a93dcd49a44afcea1dccb89a8e1=1537410755; intercom-id-e6ig86ej=c6eeb700-8385-471a-9780-6fe9306bb683; _a2_csrf=MTU0NjY4MzczMnxJbmxRVm0xWVJEVjRjRzB5VEV4NlpHRXJiMGxxUzNBM1JtZEdSV013WnprNFFuRkNlbEYxTVVkWE5VMDlJZ289fK5Dr3heKxO5PAHxoj_yUx1Wr9wZ7U9Xzl52yHlq6K7q; SL_GWPT_Show_Hide_tmp=1; SL_wptGlobTipTmp=1; BL_D_PROV=; BL_T_PROV=; csrftoken=Fj3ym8%2FFE9NSxNt4RJ5WMmGxMhdBskTo7k5sMjLjAPLeyJTH8bS1vtnr7CK%2BHHUY%2F3SyRl1gS5To7h9w36VbYQ%3D%3D',
    'x-csrftoken': 'Fj3ym8/FE9NSxNt4RJ5WMmGxMhdBskTo7k5sMjLjAPLeyJTH8bS1vtnr7CK+HHUY/3SyRl1gS5To7h9w36VbYQ==',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'content-type': 'application/json; charset=UTF-8',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'referer': 'https://goldenbee.applysquare.com/retrieval/list',
    'authority': 'goldenbee.applysquare.com',
    'dnt': '1',
}

data = {"query":"\\nquery($q: String, $year: Int\\n    $company: InputGoldenbeeReportSearchParamCompany,\\n    $offset: Int!, $limit: Int!\\n    $review:  InputGoldenbeeReportSearchParamReview){\\n    goldenbee{\\n        report{\\n            search(q:$q, company: $company,review: $review,\\n                year: $year,\\n                offset: $offset, limit: $limit){\\n                items{\\n                    doc_name\\n                    company{\\n                        name\\n                        industry\\n                        nature\\n                        scale\\n                        sub_industry\\n                        num_of_published_report\\n                    }\\n                    company_id\\n                    id\\n                    doc(q:$q){\\n                        matched_doc_paragraph\\n                        cover_url\\n                        images{\\n                            url\\n                        }\\n                    }\\n                    status\\n                    year\\n                    review{\\n                        body{\\n                            parameters{\\n                                name{\\n                                    key\\n                                    value\\n                                }\\n                                pages_num\\n                            }\\n                        }\\n                        commited_time\\n                    }\\n                }\\n                page_info{\\n                    count\\n                    offset\\n                }\\n            }\\n        }\\n    }\\n}","variables":{"company":{},"review":{},"q":urllib.parse.quote("中国航空"),"offset":0,"limit":10}}

response = requests.post('https://goldenbee.applysquare.com/ajax/graphql', headers=headers, data=data)


In [108]:
urllib.parse.quote("中国航空")

'%E4%B8%AD%E5%9B%BD%E8%88%AA%E7%A9%BA'

In [112]:
response

<Response [400]>

In [113]:
response.text

'InvalidArgument error: invalid json.:\n\t/go/src/bitbucket.org/applysquare/applysquare-go/pkg/backend/gql/handler/handler.go:115: bitbucket.org/applysquare/applysquare-go/pkg/backend/gql/handler.NewRequestOptions\n'

# 获取见微-社会责任报告列表

In [4]:
import pandas as pd
import requests, os, json 
import uncurl

In [1]:
# cmd = '''curl "https://www.jianweidata.com/data/api/TestSearch/SearchMainFiling" -H "Origin: https://www.jianweidata.com" -H "Accept-Encoding: gzip, deflate, br" -H "Accept-Language: zh-CN,zh;q=0.9" -H "User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36" -H "Content-Type: application/x-www-form-urlencoded; charset=UTF-8" -H "Accept: */*" -H "Referer: https://www.jianweidata.com/Home/SearchMainFiling" -H "X-Requested-With: XMLHttpRequest" -H "Connection: keep-alive" -H "DNT: 1" --data "query=^&label=^%^5B601666+^%^E5^%^B9^%^B3^%^E7^%^85^%^A4^%^E8^%^82^%^A1^%^E4^%^BB^%^BD^%^5D^&titleMust=^&titleShould=^%^E7^%^A4^%^BE^%^E4^%^BC^%^9A^%^E8^%^B4^%^A3^%^E4^%^BB^%^BB^%^E6^%^8A^%^A5^%^E5^%^91^%^8A^&titleMustNot=^&contentMust=^&contentShould=^&contentMustNot=^&subTitleMust=^&subTitleMustNot=^&startDate=2000^%^2F1^%^2F1^&endDate=2019^%^2F1^%^2F20^&isSimpleQuery=false^&sort=2^&token=TB656swXIGe4fjB08ryV8iTBC0Ge^%^252f^%^252b8a85gGpZatXLOjA6GM^%^252bAfVqTsObbR9XxXEnvNpfHs8f7O5wHI1w54Gjr2Qhk8uml1aYScwf8tBvpn4IuFFQzEZJUD^%^252brWp1meMAuJnxbbiiAaDgFkZ2Kw0wkh^%^252fbMyLfTDOn^&pageNum=1^&pageSize=1^&searchType=1^&sector=1^&isLockedAsset=false^&platform=web_inst^&filters^%^5B0^%^5D^%^5Bfield^%^5D=2^&filters^%^5B0^%^5D^%^5Bvalue^%^5D=20^&filters^%^5B0^%^5D^%^5Blabel^%^5D=^%^E5^%^85^%^B6^%^E4^%^BB^%^96^%^E9^%^87^%^8D^%^E5^%^A4^%^A7^%^E4^%^BA^%^8B^%^E9^%^A1^%^B9^&nearMode=0" --compressed'''
# cmd = '''curl "https://www.jianweidata.com/data/api/TestSearch/SearchMainFiling" -H "Origin: https://www.jianweidata.com" -H "Accept-Encoding: gzip, deflate, br" -H "Accept-Language: zh-CN,zh;q=0.9" -H "User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36" -H "Content-Type: application/x-www-form-urlencoded; charset=UTF-8" -H "Accept: */*" -H "Referer: https://www.jianweidata.com/Home/SearchMainFiling" -H "X-Requested-With: XMLHttpRequest" -H "Connection: keep-alive" -H "DNT: 1" --data "query=^&label=^%^5B601666+^%^E5^%^B9^%^B3^%^E7^%^85^%^A4^%^E8^%^82^%^A1^%^E4^%^BB^%^BD^%^5D^&titleMust=^&titleShould=^%^E7^%^A4^%^BE^%^E4^%^BC^%^9A^%^E8^%^B4^%^A3^%^E4^%^BB^%^BB^%^E6^%^8A^%^A5^%^E5^%^91^%^8A^&titleMustNot=^&contentMust=^&contentShould=^&contentMustNot=^&subTitleMust=^&subTitleMustNot=^&startDate=2000^%^2F1^%^2F1^&endDate=2019^%^2F1^%^2F20^&isSimpleQuery=false^&sort=2^&token=TB656swXIGe4fjB08ryV8iTBC0Ge^%^252f^%^252b8a85gGpZatXLOjA6GM^%^252bAfVqTsObbR9XxXEnvNpfHs8f7O5wHI1w54Gjr2Qhk8uml1aYScwf8tBvpn4IuFFQzEZJUD^%^252brWp1meMAuJnxbbiiAaDgFkZ2Kw0wkh^%^252fbMyLfTDOn^&pageNum=1^&pageSize=1^&searchType=1^&sector=1^&isLockedAsset=false^&platform=web_inst^&filters^%^5B0^%^5D^%^5Bfield^%^5D=2^&filters^%^5B0^%^5D^%^5Bvalue^%^5D=20^&filters^%^5B0^%^5D^%^5Blabel^%^5D=^%^E5^%^85^%^B6^%^E4^%^BB^%^96^%^E9^%^87^%^8D^%^E5^%^A4^%^A7^%^E4^%^BA^%^8B^%^E9^%^A1^%^B9^&nearMode=0" --compressed'''
# req = uncurl.parse(cmd)
# print(req)
# import curlify
# curl "https://www.baidu.com/" -H "Connection: keep-alive" -H "Cache-Control: max-age=0" -H "Upgrade-Insecure-Requests: 1" -H "User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36" -H "DNT: 1" -H "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" -H "Accept-Encoding: gzip, deflate, br" -H "Accept-Language: zh-CN,zh;q=0.9" -H "Cookie: BIDUPSID=5C47710591A7C5A52E9084DFE96C3EA2; PSTM=1528284658; sug=3; ORIGIN=0; bdime=0; BAIDUID=5C47710591A7C5A52E9084DFE96C3EA2:SL=0:NR=10:FG=1; BDUSS=FLajg4T09QZUdtLVlxM0ZXWWctcnV5WGxKWXF5WWVqR2NWNn5sb0ppOGl-eEJjQVFBQUFBJCQAAAAAAAAAAAEAAAB3xBwiv-C5z9Ch19AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACJy6VsiculbTm; MCITY=-^%^3A; BD_UPN=12314353; BD_HOME=1; BD_CK_SAM=1; PSINO=1; H_PS_PSSID=26524_1453_21105_28328_28132_26350_28267_27244_20718; delPer=1; H_PS_645EC=57f9LygJm9jT^%^2F46z7dvtv7IzOuHefQQYenKq1jj8sfSeAFlmiHgjexnwdIA; sugstore=0" --compressed
# response = requests.get("http://baidu.com")
# print(curlify.to_curl(response.request))
# curlify.to_curl(req)

In [137]:
import pandas as pd
import numpy as np
import requests, os, json

In [30]:
from Proxy import *

In [82]:
proxies = get_proxy()

成功获取代理！


{'https': 'https://183.148.142.82:9999'}

In [4]:
DF = pd.read_csv("321.csv", encoding="utf8", engine="python")
DF = DF.iloc[200:,:]

In [43]:
DFdropna = DF[~DF["股票代码"].isna()]
l = list(zip(DFdropna["股票代码"], DFdropna["A股上市名称（股票名称）"]))

In [9]:
DF

,序号,排名,企业名称,地区,是否A股上市,A股上市名称（股票名称）,股票代码,上市子公司（股票代码）,备注,分工
200,201,270,北京三聚环保新材料股份有限公司,北京,Y,三聚环保,300072,NaN,NaN,周积萍
201,202,271,江苏洋河酒厂股份有限公司,江苏,Y,洋河股份,002304,NaN,NaN,NaN
202,203,273,五菱汽车集团控股有限公司,广西,N（港股00305）,NaN,NaN,NaN,NaN,NaN
203,204,274,福耀玻璃工业集团股份有限公司,福建,Y,福耀玻璃,600660,NaN,NaN,NaN
204,205,275,内蒙古鄂尔多斯资源股份有限公司,内蒙古,Y,鄂尔多斯,600295,NaN,NaN,NaN
205,206,276,山东恒邦冶炼股份有限公司,山东,Y,恒邦股份,002237,NaN,NaN,NaN
206,207,277,唐山三友化工股份有限公司,河北,Y,三友化工,600409,NaN,NaN,NaN
207,208,279,瑞康医药股份有限公司,山东,Y,瑞康医药,002589,NaN,NaN,NaN
208,209,280,云南铝业股份有限公司,云南,Y,云铝股份,000807,NaN,NaN,NaN
209,210,281,宝胜科技创新股份有限公司,江苏,Y,宝胜股份,600973,NaN,NaN,NaN


In [28]:
# code = "601666"
# corp = "平煤股份"
code = DF["股票代码"].iloc[0]
corp = DF["A股上市名称（股票名称）"].iloc[0]

In [142]:
def getReport2017(param):
    code, corp = param
    query = '''query=&label=%5B601666+%E5%B9%B3%E7%85%A4%E8%82%A1%E4%BB%BD%5D&titleMust=&titleShould=%E7%A4%BE%E4%BC%9A%E8%B4%A3%E4%BB%BB%E6%8A%A5%E5%91%8A&titleMustNot=&contentMust=&contentShould=&contentMustNot=&subTitleMust=&subTitleMustNot=&startDate=2000%2F1%2F1&endDate=2019%2F1%2F20&isSimpleQuery=false&sort=2&token=TB656swXIGe4fjB08ryV8iTBC0Ge%252f%252b8a85gGpZatXLOjA6GM%252bAfVqTsObbR9XxXEnvNpfHs8f7O5wHI1w54Gjr2Qhk8uml1aYScwf8tBvpn4IuFFQzEZJUD%252brWp1meMAuJnxbbiiAaDgFkZ2Kw0wkh%252fbMyLfTDOn&pageNum=1&pageSize=1&searchType=1&sector=1&isLockedAsset=false&platform=web_inst&filters%5B0%5D%5Bfield%5D=2&filters%5B0%5D%5Bvalue%5D=20&filters%5B0%5D%5Blabel%5D=%E5%85%B6%E4%BB%96%E9%87%8D%E5%A4%A7%E4%BA%8B%E9%A1%B9&nearMode=0'''
    data = dict([item.split("=") for item in query.split("&")])

    token = '''
    TB656swXIGe4fjB08ryV8iTBC0Ge%2f%2b8a85gGpZatXLOjA6GM%2bAfVqTsObbR9XxXERpNUqODKzgDJHiQ4sllPFH%2fo9SehALfZtaGShbo%2fj%2btZFMUa1HeANCx8a%2f0qvlO%2fDASPlfCXD59UmP8SIR4oCISCjAArdQmX
    '''
    token = token.strip()

    data["label"] = "[{} {}]".format(code, corp)
    data["titleShould"] = "社会责任报告"
    data["filters[0][label]"] = "其他重大事项"
    data["token"] = token
    failed = []
    cond = False
    while not cond:
        try:
            res = requests.post("https://www.jianweidata.com/data/api/TestSearch/SearchMainFiling",
                data=data,
                headers={
                    "Accept": "*/*",
                    "Accept-Encoding": "gzip, deflate, br",
                    "Accept-Language": "zh-CN,zh;q=0.9",
                    "Connection": "keep-alive",
                    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
                    "DNT": "1",
                    "Origin": "https://www.jianweidata.com",
                    "Referer": "https://www.jianweidata.com/Home/SearchMainFiling",
                    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
                    "X-Requested-With": "XMLHttpRequest"
                },
                proxies=gen_proxy(proxies)
            )
            cond = res.status_code == 200
            print("{} {} done~~~".format(code, corp))
#             if not cond: print("failed...")
        except requests.exceptions.ProxyError: proxy = gen_proxy(proxies); print("{} {} 代理失败".format(code, corp))
        except: return param, ""
#     return res
    content = json.loads(res.text)
#     return content
    # urlOnline = "https://www.jianweidata.com{}".format(content["Hits"][0]["Source"]["Href"]) # 报告在线网址
    urlOnline = ["https://www.jianweidata.com{}".format(item["Source"]["Href"]) for item in content["Hits"]]

    # title = content["Hits"][0]["Highlight"]["Title"][0] # 报告标题
    # title = title.replace("<em>", "").replace("</em>", "")
    title = [item["Highlight"]["Title"][0].replace("<em>", "").replace("</em>", "") for item in content["Hits"]] # 报告标题

    # urlDownload = content["Hits"][0]["Source"]["Url"] # 报告下载链接
    urlDownload = [item["Source"]["Url"] for item in content["Hits"]] # 报告下载链接

    df = pd.DataFrame({"code": code, "corp": corp, "title": title, 
                       "reportDownload": urlDownload, "reportOnline": urlOnline})
#     return df    
    if sum(df.shape)>0:
        index = df.title.map(lambda x: str(x).find("2017年")>=0)
        if np.any(index):
            return param, df[index]
        else: return param, None
    else:
        return param, None


In [76]:
from multiprocessing import Pool

In [143]:
c = getReport2017(l[4])

002237 恒邦股份 done~~~


In [144]:
c

(('002237', '恒邦股份'), None)

In [149]:
len(l)

67

In [150]:
# dfs = [getReport2017(item) for item in l[:3]]
# dfs

### 使用多进程

pool = Pool()
res = pool.imap_unordered(getReport2017, l)
dfs = [item for item in res]
pool.close()

600295 鄂尔多斯 done~~~
300072 三聚环保 done~~~
002304 洋河股份 done~~~
600660 福耀玻璃 done~~~
002237 恒邦股份 done~~~
000807 云铝股份 done~~~
600409 三友化工 done~~~
002589 瑞康医药 done~~~
600973 宝胜股份 done~~~
601666 平煤股份 done~~~
000021 深科技 done~~~
000060 中金岭南 done~~~
600535 天士力 done~~~
002475 立讯精密 代理失败
600141 兴发集团 done~~~
002475 立讯精密 done~~~
600565 迪马股份 done~~~
600219 南山铝业 代理失败
600219 南山铝业 done~~~
600801 华新水泥 代理失败
600801 华新水泥 done~~~
000572 海马汽车 done~~~
000401 冀东水泥 代理失败
000401 冀东水泥 done~~~
002426 胜利精密 代理失败
002048 宁波华翔 代理失败
603288 海天味业 代理失败
600079 人福医药 代理失败
002426 胜利精密 代理失败
002426 胜利精密 done~~~
603858 步长制药 done~~~
002048 宁波华翔 代理失败
002048 宁波华翔 done~~~
600085 同仁堂 done~~~
603288 海天味业 代理失败
603288 海天味业 done~~~
600079 人福医药 代理失败
600742 一汽富维 done~~~
600079 人福医药 done~~~
600623 华谊集团 done~~~
600567 山鹰纸业 代理失败
002600 领益制造 代理失败
600997 开滦股份 代理失败
000729 燕京啤酒 代理失败
600567 山鹰纸业 代理失败
600567 山鹰纸业 done~~~
601179 中国西电 done~~~
000830 鲁西化工 done~~~
601005 重庆钢铁 done~~~
600507 方大特钢 done~~~
002082 万邦德 done~~~
000698 沈阳化工 代理失败
002600 领益制造 代理失败
0

In [161]:
dfs[0][1] !=None

False

In [169]:
fore = [item[0] for item in dfs]

In [171]:
back = np.array([item[1] for item in dfs])

In [180]:
def dropNone(x):
    return (x[1] is not None) or (x[1] == "")

In [184]:
data = np.array(list(filter(dropNone, dfs)))

In [189]:
dfDownload = pd.concat(data[:, 1], ignore_index=True)

In [190]:
dfDownload.to_csv("dfDownload.csv", encoding="gbk", index=False)

In [212]:
len("")

0

In [213]:
def dropEmptyString(x):
    if x[1] is None:
        return False
    elif len(x[1])==1:
        return False
    else:
        return x[1] == ""

In [214]:
data2 = np.array(list(filter(dropEmptyString, dfs)))

In [217]:
dfDownload.shape

(28, 5)

In [223]:
def countNone(x):
    return x[1] is None

listNone = list(filter(countNone, dfs))
data2 = np.array(listNone)

In [254]:
a=data2[:,0]
a = list(zip(*a))

In [255]:
a

[('600295',
  '002237',
  '000807',
  '002589',
  '600565',
  '600801',
  '000572',
  '002426',
  '603858',
  '002048',
  '603288',
  '600742',
  '600079',
  '600567',
  '601179',
  '000830',
  '002082',
  '002600',
  '000666',
  '300025',
  '000632',
  '603225',
  '002737',
  '002358',
  '002756',
  '601717',
  '600426',
  '000698',
  '600577',
  '603298',
  '600320',
  '601311',
  '000902',
  '002032',
  '600781',
  '603031',
  '002309',
  '600169',
  '000887'),
 ('鄂尔多斯',
  '恒邦股份',
  '云铝股份',
  '瑞康医药',
  '迪马股份',
  '华新水泥',
  '海马汽车',
  '胜利精密',
  '步长制药',
  '宁波华翔',
  '海天味业',
  '一汽富维',
  '人福医药',
  '山鹰纸业',
  '中国西电',
  '鲁西化工',
  '万邦德',
  '领益制造',
  '经纬纺机',
  '华星创业',
  '三木集团',
  '新凤鸣',
  '葵花药业',
  '森源电气',
  '永兴特钢',
  '郑煤机',
  '华鲁恒升',
  '沈阳化工',
  '精达股份',
  '杭叉集团',
  '振华重工',
  '骆驼股份',
  '新洋丰',
  '苏泊尔',
  '辅仁药业',
  '安德利',
  '中利集团',
  '太原重工',
  '中鼎股份')]

In [220]:
len(listNone)

39

In [222]:
len(l) == 28+39 # 67

True

In [256]:
dfNone = pd.DataFrame({"None_code": a[0], "None_corp": a[1]})
dfNone

,None_code,None_corp
0,600295,鄂尔多斯
1,002237,恒邦股份
2,000807,云铝股份
3,002589,瑞康医药
4,600565,迪马股份
5,600801,华新水泥
6,000572,海马汽车
7,002426,胜利精密
8,603858,步长制药
9,002048,宁波华翔


In [257]:
dfNone.to_csv("dfNone.csv", encoding="gbk", index=False)

In [89]:
# pd.concat([parse_(item) for item in dfs], ignore_index=True)

,code,corp,title,urlDownload,urlOnline
0,300072,三聚环保,洋河股份：2017年度社会责任报告,http://www.cninfo.com.cn/finalpage/2018-04-27/...,https://www.jianweidata.com/Home/FilingDetail?...
1,300072,三聚环保,福耀玻璃：2017年度社会责任报告,http://www.cninfo.com.cn/finalpage/2018-03-17/...,https://www.jianweidata.com/Home/FilingDetail?...
2,300072,三聚环保,三聚环保：2017年社会责任报告,http://www.cninfo.com.cn/finalpage/2018-04-28/...,https://www.jianweidata.com/Home/FilingDetail?...


In [88]:
def parse_(res):
    content = json.loads(res.text)
    # urlOnline = "https://www.jianweidata.com{}".format(content["Hits"][0]["Source"]["Href"]) # 报告在线网址
    urlOnline = ["https://www.jianweidata.com{}".format(item["Source"]["Href"]) for item in content["Hits"]]

    # title = content["Hits"][0]["Highlight"]["Title"][0] # 报告标题
    # title = title.replace("<em>", "").replace("</em>", "")
    title = [item["Highlight"]["Title"][0].replace("<em>", "").replace("</em>", "") for item in content["Hits"]] # 报告标题

    # urlDownload = content["Hits"][0]["Source"]["Url"] # 报告下载链接
    urlDownload = [item["Source"]["Url"] for item in content["Hits"]] # 报告下载链接

    df = pd.DataFrame({"code": code, "corp": corp, "title": title, "urlDownload": urlDownload, "urlOnline": urlOnline})
    index = df.title.str.find("2017年")>=0
    return df[index]

In [75]:
index = df.title.str.find("2017年")>=0
df = df[index]
df.iloc[:, 2:]

,title,urlDownload,urlOnline
0,福耀玻璃：2017年度社会责任报告,http://www.cninfo.com.cn/finalpage/2018-03-17/...,https://www.jianweidata.com/Home/FilingDetail?...


# 下载报告

In [136]:
import os

In [89]:
Path = r".\年度报告\download"

In [90]:
file = requests.get(urlDownload)

In [96]:
with open(r"{}\{}.pdf".format(Path, title), "wb") as f:
    f.write(file.content)

In [101]:
import requests

headers = {
    'origin': 'https://goldenbee.applysquare.com',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'x-requested-with': 'XMLHttpRequest',
    'cookie': 'UM_distinctid=165f4d269c367d-0c70e71d1ccba6-323b5b03-144000-165f4d269c4a29; Hm_lvt_50058a93dcd49a44afcea1dccb89a8e1=1537410755; intercom-id-e6ig86ej=c6eeb700-8385-471a-9780-6fe9306bb683; _a2_csrf=MTU0NjY4MzczMnxJbmxRVm0xWVJEVjRjRzB5VEV4NlpHRXJiMGxxUzNBM1JtZEdSV013WnprNFFuRkNlbEYxTVVkWE5VMDlJZ289fK5Dr3heKxO5PAHxoj_yUx1Wr9wZ7U9Xzl52yHlq6K7q; csrftoken=pSlhxNwg8bJQ9LufmsOAz6gicF08fC8cZySH8gCRoHNt3AeY4lFX39vbjMVgQaPlNufwDCCuIGBhhPSw7df74A%3D%3D',
    'x-csrftoken': 'pSlhxNwg8bJQ9LufmsOAz6gicF08fC8cZySH8gCRoHNt3AeY4lFX39vbjMVgQaPlNufwDCCuIGBhhPSw7df74A==',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'content-type': 'application/json; charset=UTF-8',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'referer': 'https://goldenbee.applysquare.com/retrieval/list',
    'authority': 'goldenbee.applysquare.com',
    'dnt': '1',
}

data = '{"query":"\\nquery($q: String, $year: Int\\n    $company: InputGoldenbeeReportSearchParamCompany,\\n    $offset: Int!, $limit: Int!\\n    $review:  InputGoldenbeeReportSearchParamReview){\\n    goldenbee{\\n        report{\\n            search(q:$q, company: $company,review: $review,\\n                year: $year,\\n                offset: $offset, limit: $limit){\\n                items{\\n                    doc_name\\n                    company{\\n                        name\\n                        industry\\n                        nature\\n                        scale\\n                        sub_industry\\n                        num_of_published_report\\n                    }\\n                    company_id\\n                    id\\n                    doc(q:$q){\\n                        matched_doc_paragraph\\n                        cover_url\\n                        images{\\n                            url\\n                        }\\n                    }\\n                    status\\n                    year\\n                    review{\\n                        body{\\n                            parameters{\\n                                name{\\n                                    key\\n                                    value\\n                                }\\n                                pages_num\\n                            }\\n                        }\\n                        commited_time\\n                    }\\n                }\\n                page_info{\\n                    count\\n                    offset\\n                }\\n            }\\n        }\\n    }\\n}","variables":{"company":{},"review":{},"q":"\u5E73\u9876\u5C71\u5929\u5B89\u7164\u4E1A\u80A1\u4EFD\u6709\u9650\u516C\u53F8","offset":0,"limit":10}}'

response = requests.post('https://goldenbee.applysquare.com/ajax/graphql', headers=headers, data=data.encode("utf8"))


In [104]:
report = json.loads(response.text)

In [116]:
report["data"]["goldenbee"]["report"]["search"]["items"]

[{'company': {'industry': 'caijue',
   'name': '平顶山天安煤业股份有限公司',
   'nature': ['guoyoukonggu'],
   'num_of_published_report': 'range_5_10',
   'scale': 'medium',
   'sub_industry': None},
  'company_id': 'cnXpuCBBH',
  'doc': {'cover_url': 'https://hhqraxfd5iza7qpzwd5.exp.bcedocument.com/target/doc-ifjsd0z9dgh3sch/doc-ifjsd0z9dgh3sch.jpg',
   'images': [],
   'matched_doc_paragraph': None},
  'doc_name': 'B45-平顶山天安煤业股份有限公司.PDF',
  'id': 'pG7oJgIiM',
  'review': {'body': {'parameters': {'name': {'key': 'shehuizeren',
      'value': None},
     'pages_num': 'range_11_30'}},
   'commited_time': '2018-08-30T12:22:54.524Z'},
  'status': 'completed',
  'year': 2018},
 {'company': {'industry': 'caijue',
   'name': '平顶山天安煤业股份有限公司',
   'nature': ['guoyoukonggu'],
   'num_of_published_report': 'range_5_10',
   'scale': 'medium',
   'sub_industry': None},
  'company_id': 'cnXpuCBBH',
  'doc': None,
  'doc_name': 'b40-平顶山天安煤业股份有限公司.pdf',
  'id': 'cmg3e6jeu',
  'review': {'body': {'parameters': {'na